In [0]:
import json
from pathlib import Path
from functions import get_function

# Variables
color                       = dbutils.widgets.get("color")
job_settings                = json.loads(dbutils.widgets.get("job_settings"))
table                       = job_settings['table']
settings                    = json.loads(next(Path().glob(f"./layer_*_{color}/{table}.json")).read_text())
dst_table_name              = settings["dst_table_name"]

# Print job and table settings
settings_message = f"\n\nDictionary from {color}_settings.json:\n\n"
settings_message += json.dumps(job_settings, indent=4)
settings_message += f"\n\nContents of {table}.json:\n\n"
settings_message += json.dumps(settings, indent=4)
print(settings_message)

# One function for pipeline
if "pipeline_function" in settings:
    pipeline_function = get_function(settings["pipeline_function"])
    pipeline_function(spark, settings)

# Individual functions for each step
elif all(k in settings for k in ["read_function", "transform_function", "write_function"]):
    read_function = get_function(settings["read_function"])
    transform_function = get_function(settings["transform_function"])
    write_function = get_function(settings["write_function"])

    df = read_function(spark, settings)
    df = transform_function(spark, settings, df)
    write_function(spark, settings, df)
else:
    raise Exception(f"Could not find any ingest function name in settings.")